# Video Frame Search System with BLIP & Pinecone

This notebook sets up a complete video semantic search engine that:
- Extracts frames from videos
- Generates captions using BLIP
- Stores embeddings in Pinecone
- Enables natural language search

---


 ## Step 1: Setup - Clone Repository & Install Dependencies



In [14]:
# Clone the repository
!git clone https://github.com/pranavacchu/capstone-BLIP.git
%cd capstone-BLIP

# Install dependencies
print("📦 Installing dependencies... This will take 3-5 minutes")
!pip install -q opencv-python-headless pillow numpy pandas tqdm python-dotenv
!pip install -q torch torchvision transformers sentence-transformers
!pip install -q pinecone FlagEmbedding

print("\n✅ Installation complete!")

# Check GPU availability
import torch
if torch.cuda.is_available():
    print(f"\n🚀 GPU detected: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("\n⚠️ No GPU detected. Using CPU (slower but works)")

Cloning into 'capstone-BLIP'...
remote: Enumerating objects: 136, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 136 (delta 83), reused 93 (delta 40), pack-reused 0 (from 0)
Receiving objects: 100% (136/136), 189.59 KiB | 6.32 MiB/s, done.
Resolving deltas: 100% (83/83), done.
/content/capstone-BLIP/capstone-BLIP/capstone-BLIP/capstone-BLIP
📦 Installing dependencies... This will take 3-5 minutes

✅ Installation complete!

🚀 GPU detected: Tesla T4
   Memory: 15.8 GB


## Step 1b (Optional): Install Object Detection Dependencies

Install additional dependencies for object detection pipeline.

In [15]:
# Install Grounding DINO dependencies
print("Installing Grounding DINO and dependencies...")
print("This may take 2-3 minutes...")

import subprocess
subprocess.run(['pip', 'install', '-q', 'timm'], check=False)
subprocess.run(['pip', 'install', '-q', 'supervision'], check=False)

print("\nGrounding DINO dependencies installed!")
print("Models will be downloaded automatically from Hugging Face on first use")

Installing Grounding DINO and dependencies...
This may take 2-3 minutes...

Grounding DINO dependencies installed!
Models will be downloaded automatically from Hugging Face on first use


## Step 2: Configure Pinecone API Key

Enter your Pinecone credentials below:
- **API Key**: Your Pinecone API key
- **Index Host**: Your index URL (from Pinecone dashboard)

Your current settings:
```
API Key: pcsk_51Fgoo_2S9NQf4CHi8LMpX7AXKv4TEHgRdXR3huZcCwBdJkr7BMvmdGHeRASrk5hkz4AH1
Host: https://capstone-b5a0x4x.svc.aped-4627-b74a.pinecone.io
```

In [16]:
import os

# Set your Pinecone credentials
PINECONE_API_KEY = "pcsk_51Fgoo_2S9NQf4CHi8LMpX7AXKv4TEHgRdXR3huZcCwBdJkr7BMvmdGHeRASrk5hkz4AH1"
PINECONE_HOST = "https://capstone-b5a0x4x.svc.aped-4627-b74a.pinecone.io"
PINECONE_ENVIRONMENT = "us-east-1"

# Write to .env file
with open('.env', 'w') as f:
    f.write(f"PINECONE_API_KEY={PINECONE_API_KEY}\n")
    f.write(f"PINECONE_HOST={PINECONE_HOST}\n")
    f.write(f"PINECONE_ENVIRONMENT={PINECONE_ENVIRONMENT}\n")

print("✅ Configuration saved!")

✅ Configuration saved!


##  Step 3: Test Connection to Pinecone



In [17]:
from video_search_engine import VideoSearchEngine

print("🔌 Connecting to Pinecone...")
engine = VideoSearchEngine()

# Get database stats
stats = engine.get_index_stats()

print("\n✅ Successfully connected to Pinecone!")
print(f"\n📊 Database Statistics:")
print(f"   Index: {stats.get('index_name', 'N/A')}")
print(f"   Total vectors: {stats.get('total_vectors', 0):,}")
print(f"   Dimension: {stats.get('dimension', 'N/A')}")
print(f"   Capacity: {stats.get('capacity', 'N/A')}")

# Optional: Show namespace statistics if available
if stats.get('namespaces'):
    print(f"\n📁 Namespaces:")
    for namespace, ns_info in stats.get('namespaces', {}).items():
        # Handle NamespaceSummary object - access vector_count attribute
        vector_count = ns_info.vector_count if hasattr(ns_info, 'vector_count') else ns_info
        print(f"   - {namespace}: {vector_count:,} vectors")


🔌 Connecting to Pinecone...

✅ Successfully connected to Pinecone!

📊 Database Statistics:
   Index: test
   Total vectors: 250
   Dimension: 1024
   Capacity: Serverless

📁 Namespaces:
   - videos:2025-11-06:coat_jacket: 1 vectors
   - folder: 2 vectors
   - backpack: 2 vectors
   - : 235 vectors
   - videos:2025-11-06:folder: 1 vectors
   - duffel_bag: 2 vectors
   - coat_jacket: 3 vectors
   - videos:2025-11-06:duffel_bag: 2 vectors
   - videos:2025-11-06:backpack: 2 vectors


## 🚀 Step 4 (Optional): Start API Server for React Integration

**Run this cell ONLY if you want to use the React frontend.**

This cell:
- Authenticates with ngrok (requires free account at https://ngrok.com)
- Starts a FastAPI server that runs on Colab's GPU
- Creates a public URL you can use from your React app

**⚠️ Important:**
1. You need a free ngrok account: https://dashboard.ngrok.com/signup
2. Get your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken
3. Replace the NGROK_AUTH_TOKEN below with your token
4. Keep this cell running while using the React app
5. Copy the ngrok URL that appears and paste it in your React app

**Instructions for React App:**
1. Open terminal in your React project folder
2. Run: `npm run dev`
3. Navigate to: `http://localhost:5173/dashboard`
4. Click on the "🔍 Video Search" or "📤 Process Video" tab
5. Paste the ngrok URL in the connection section and click "Connect"
6. Start using the integrated dashboard!

**Note:** If you just want to use the notebook directly (without React), skip this cell and continue to Step 5.

In [ ]:
# Install FastAPI and dependencies
!pip install -q fastapi uvicorn pyngrok nest-asyncio python-multipart

# Authenticate ngrok
from pyngrok import ngrok

# Set your ngrok authtoken (get it from: https://dashboard.ngrok.com/get-started/your-authtoken)
NGROK_AUTH_TOKEN = "2uq19SSTYxlDhPoiMXnJXHuKAUV_6w77sQcS4KX3fRnDr1c4W"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
print("✅ ngrok authenticated successfully!")

# Start the API server with ngrok tunnel
import nest_asyncio
import uvicorn
from colab_api_server import app

nest_asyncio.apply()

print("\n🌐 Starting ngrok tunnel on port 8000...")
print("=" * 80)

# Kill any existing tunnels
ngrok.kill()

# Create ngrok tunnel
public_url_obj = ngrok.connect(8000, bind_tls=True)
# Extract the public URL string
public_url = str(public_url_obj).split('"')[1] if '"' in str(public_url_obj) else public_url_obj.public_url

print(f"\n✅ COLAB API SERVER READY")
print("=" * 80)
print(f"\n🌍 Public URL: {public_url}")
print(f"\n📋 Copy this URL and paste it into your React app:")
print(f"   {public_url}")
print("\n" + "=" * 80)

# Initialize the video search engine
print("\n🔧 Initializing Video Search Engine...")
from video_search_engine import VideoSearchEngine

# Create a global engine instance for the app to use
import colab_api_server
if not colab_api_server.engine:
    colab_api_server.engine = VideoSearchEngine()
    print("✅ Engine initialized successfully!")
else:
    print("✅ Engine already initialized!")

# =========================================================================
# 🚀 PRE-LOAD EMBEDDING MODEL (Avoids 2.24GB download on first search)
# =========================================================================
print("\n🔄 Pre-loading embedding model...")
print("   This will download ~2.24GB (one-time, takes 2-3 minutes)")
print("   Future searches will be FAST!")

try:
    from embedding_generator import EmbeddingGenerator
    temp_embedder = EmbeddingGenerator()
    # Trigger model download and initialization
    _ = temp_embedder.generate_embedding("warmup")
    print("✅ Embedding model ready! Searches will now be fast.")
except Exception as e:
    print(f"⚠️  Model pre-load failed: {e}")
    print("   First search will still work but will be slower")

# =========================================================================

# Display GPU status
import torch
if torch.cuda.is_available():
    print(f"\n🚀 GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("\n⚠️  Running on CPU (slower)")

print("\n🚀 Starting FastAPI server...")
print("   Keep this notebook running while using the React app")
print("   The server will process videos using Colab's GPU")
print("\n" + "=" * 80)
print("\n📡 Server is ready to accept requests!")
print("   Go to your React app and paste the URL above to connect")
print("\n" + "=" * 80 + "\n")

# Start the server (this will block and keep running)
# Python 3.12+ compatible approach - run server directly with asyncio
import asyncio

async def start_server():
    """Start the FastAPI server using uvicorn"""
    config = uvicorn.Config(
        app,
        host="0.0.0.0",
        port=8000,
        log_level="info"
    )
    server = uvicorn.Server(config)
    await server.serve()

# Run the server
asyncio.run(start_server())

## Step 5: Upload a Video File

## Step 6: Choose Captioning Method

Select between standard BLIP or object detection + BLIP pipeline.

In [18]:
from google.colab import files
import os
import subprocess
from urllib.parse import urlparse, parse_qs

print("📤 Choose how to get your video:\n")
print("1. Upload from computer (recommended for small files < 100MB)")
print("2. Download from URL (direct video file)")
print("3. Download from YouTube URL\n")

choice = input("Enter choice (1/2/3): ").strip()
video_path = None

if choice == "1":
    print("\n📁 Please select your video file...")
    uploaded = files.upload()
    if uploaded:
        video_path = list(uploaded.keys())[0]
        print(f"✅ Uploaded: {video_path}")
    else:
        print("❌ No file uploaded")

elif choice == "2":
    video_url = input("\nEnter video URL (direct link to .mp4, .avi, etc.): ").strip()

    if not video_url:
        print("❌ No URL provided")
    else:
        # Extract filename from URL or use default
        parsed_url = urlparse(video_url)
        url_filename = os.path.basename(parsed_url.path)

        # Use URL filename if it has an extension, otherwise use default
        if url_filename and '.' in url_filename:
            video_filename = url_filename
        else:
            video_filename = "downloaded_video.mp4"

        print(f"⬇️ Downloading from URL...")
        print(f"   Target file: {video_filename}")

        try:
            # Use subprocess for better control
            result = subprocess.run(
                ['wget', '-O', video_filename, video_url, '--no-check-certificate', '-q', '--show-progress'],
                capture_output=True,
                text=True,
                timeout=300
            )

            if result.returncode == 0 and os.path.exists(video_filename):
                if os.path.getsize(video_filename) > 0:
                    video_path = video_filename
                    print(f"✅ Downloaded successfully: {video_filename}")
                else:
                    print(f"❌ Download failed: File is empty")
                    if os.path.exists(video_filename):
                        os.remove(video_filename)
            else:
                print(f"❌ Download failed: wget returned code {result.returncode}")
                # Try alternative method with curl
                print("\n🔄 Trying alternative download method (curl)...")
                result2 = subprocess.run(
                    ['curl', '-L', '-o', video_filename, video_url, '--silent', '--show-error'],
                    capture_output=True,
                    text=True,
                    timeout=300
                )

                if result2.returncode == 0 and os.path.exists(video_filename) and os.path.getsize(video_filename) > 0:
                    video_path = video_filename
                    print(f"✅ Downloaded successfully with curl: {video_filename}")
                else:
                    print(f"❌ Alternative download also failed")
                    print("   Please check if the URL is accessible and try again")

        except subprocess.TimeoutExpired:
            print("❌ Download timed out (>5 minutes). File may be too large.")
        except Exception as e:
            print(f"❌ Download error: {e}")

elif choice == "3":
    youtube_url = input("\nEnter YouTube URL (video or shorts): ").strip()

    if not youtube_url:
        print("❌ No URL provided")
    else:
        print("⬇️ Downloading from YouTube...")
        print("   Installing/updating yt-dlp...")

        # Install/update yt-dlp to latest version (fixes bot detection)
        subprocess.run(['pip', 'install', '-U', 'yt-dlp'], check=False)

        video_filename = "youtube_video.mp4"

        try:
            print(f"   Fetching video (this may take a minute)...")

            # Enhanced yt-dlp command with multiple bot detection workarounds
            result = subprocess.run(
                [
                    'yt-dlp',
                    # Format selection - prefer lower quality for faster download
                    '-f', 'best[height<=720][ext=mp4]/best[ext=mp4]/best',
                    '-o', video_filename,
                    '--no-playlist',
                    # Bot detection workarounds
                    '--extractor-args', 'youtube:player_client=android,web',
                    '--user-agent', 'Mozilla/5.0 (Linux; Android 11) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.120 Mobile Safari/537.36',
                    # Additional options
                    '--no-check-certificate',
                    '--no-warnings',
                    '--prefer-insecure',
                    # Progress and retries
                    '--progress',
                    '--retries', '3',
                    '--fragment-retries', '3',
                    youtube_url
                ],
                capture_output=True,
                text=True,
                timeout=600  # 10 minute timeout
            )

            if result.returncode == 0 and os.path.exists(video_filename):
                if os.path.getsize(video_filename) > 0:
                    video_path = video_filename
                    print(f"✅ Downloaded successfully: {video_filename}")
                else:
                    print(f"❌ Download failed: File is empty")
                    if os.path.exists(video_filename):
                        os.remove(video_filename)
            else:
                print(f"❌ YouTube download failed")
                if result.stderr:
                    error_msg = result.stderr[:500]
                    print(f"   Error details: {error_msg}")

                # Try alternative method with mobile client only
                print("\n🔄 Trying alternative method (mobile client)...")
                result2 = subprocess.run(
                    [
                        'yt-dlp',
                        '-f', 'worst[ext=mp4]/worst',  # Use lowest quality for testing
                        '-o', video_filename,
                        '--extractor-args', 'youtube:player_client=android',
                        '--user-agent', 'com.google.android.youtube/17.31.35 (Linux; U; Android 11) gzip',
                        '--no-check-certificate',
                        '--no-playlist',
                        youtube_url
                    ],
                    capture_output=True,
                    text=True,
                    timeout=300
                )

                if result2.returncode == 0 and os.path.exists(video_filename) and os.path.getsize(video_filename) > 0:
                    video_path = video_filename
                    print(f"✅ Downloaded successfully with alternative method: {video_filename}")
                else:
                    print(f"❌ Alternative method also failed")
                    print("\n💡 Troubleshooting tips:")
                    print("   1. Make sure the video is public and not age-restricted")
                    print("   2. Try using Option 1 to upload the video manually")
                    print("   3. Alternative: Download video on your computer first, then upload")
                    print("   4. Some YouTube videos may be restricted in certain regions")

        except subprocess.TimeoutExpired:
            print("❌ Download timed out (>10 minutes).")
            print("   Video may be too long or connection is slow.")
        except Exception as e:
            print(f"❌ Download error: {e}")

else:
    print("⚠️ Invalid choice. Please choose option 1, 2, or 3.")

# Validate the video file
if video_path:
    if os.path.exists(video_path):
        file_size = os.path.getsize(video_path) / (1024*1024)  # MB
        print(f"\n📹 Video ready: {video_path} ({file_size:.1f} MB)")

        # Verify it's a valid video file
        import cv2
        cap = cv2.VideoCapture(video_path)
        if cap.isOpened():
            fps = cap.get(cv2.CAP_PROP_FPS)
            frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            duration = frame_count / fps if fps > 0 else 0
            print(f"   Duration: {duration:.1f} seconds")
            print(f"   FPS: {fps:.1f}")
            print(f"   Total frames: {frame_count:,}")
            cap.release()
        else:
            print("\n⚠️ Warning: Unable to read video file. It may be corrupted.")
            print("   Please try a different video or URL.")
            video_path = None
    else:
        print(f"\n❌ Error: File not found at {video_path}")
        video_path = None

if not video_path:
    print("\n❌ No valid video file available. Please run this cell again.")

📤 Choose how to get your video:

1. Upload from computer (recommended for small files < 100MB)
2. Download from URL (direct video file)
3. Download from YouTube URL

Enter choice (1/2/3): 3

Enter YouTube URL (video or shorts): https://www.youtube.com/shorts/QhlroYnundk
⬇️ Downloading from YouTube...
   Installing/updating yt-dlp...
   Fetching video (this may take a minute)...
✅ Downloaded successfully: youtube_video.mp4

📹 Video ready: youtube_video.mp4 (0.7 MB)
   Duration: 7.6 seconds
   FPS: 30.0
   Total frames: 228


In [19]:
print("Choose your captioning method:\n")
print("1. Standard BLIP (faster, general scene captions)")
print("2. Object Detection + BLIP (slower, object-focused)")
print()

method_choice = input("Enter choice (1/2, default=1): ").strip() or "1"
use_object_detection = (method_choice == "2")

if use_object_detection:
    print("\nUsing Object Detection + BLIP pipeline")
    print("   Detects objects: bags, laptops, helmets, phones, etc.")
else:
    print("\nUsing Standard BLIP captioning")

Choose your captioning method:

1. Standard BLIP (faster, general scene captions)
2. Object Detection + BLIP (slower, object-focused)

Enter choice (1/2, default=1): 2

Using Object Detection + BLIP pipeline
   Detects objects: bags, laptops, helmets, phones, etc.


## Step 7: Select Video Recording Date

Choose the date when this video was recorded. This enables efficient date-based filtering.

In [20]:
from datetime import date, datetime

# Manual date input (since ipywidgets may not work in all Colab versions)
print("📅 Select video recording date")
print("=" * 60)
print(f"Today's date: {date.today()}")
print()

use_today = input("Use today's date? (y/n, default=y): ").strip().lower()

if use_today == 'n':
    date_input = input("Enter date (YYYY-MM-DD): ").strip()
    try:
        # Validate date format
        datetime.strptime(date_input, "%Y-%m-%d")
        video_date = date_input
    except ValueError:
        print("⚠️ Invalid date format. Using today's date.")
        video_date = date.today().strftime("%Y-%m-%d")
else:
    video_date = date.today().strftime("%Y-%m-%d")

print(f"\n✅ Video date set to: {video_date}")
print(f"   This will be used for efficient date-based searching")


📅 Select video recording date
Today's date: 2025-11-10

Use today's date? (y/n, default=y): y

✅ Video date set to: 2025-11-10
   This will be used for efficient date-based searching


## Step 8: Process the Video

This will:
1. Extract frames from the video (removing redundant frames)
2. Generate captions using BLIP AI model
3. Create embeddings for semantic search
4. Upload to Pinecone database

**Expected time:**
- 1 minute video: ~2-3 minutes with GPU
- 5 minute video: ~8-10 minutes with GPU
- CPU mode: 3-5x slower

In [21]:
import time
from datetime import datetime

if 'video_path' not in locals() or not video_path:
    print("❌ Please upload a video first (run the previous cell)")
else:
    # Set video name
    video_name = input("Enter a name for this video (or press Enter for auto-name): ").strip()
    if not video_name:
        video_name = f"video_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

    print(f"\n🎬 Processing video: {video_name}")
    print(f"📅 Video date: {video_date}")
    print("⏳ This will take a few minutes... Please wait.\n")
    print("=" * 80)

    start_time = time.time()

    try:
        # Process the video with improved logging
        stats = engine.process_video(
            video_path=video_path,
            video_name=video_name,
            video_date=video_date,  # Include video date
            save_frames=False,  # Set to True to save frames
            upload_to_pinecone=True,
            use_object_detection=use_object_detection  # Use selected method
        )

        processing_time = time.time() - start_time

        print("\n" + "=" * 80)
        print("\n✅ VIDEO PROCESSING COMPLETE!\n")
        print(f"📊 Processing Statistics:")
        print(f"   Video name: {video_name}")
        print(f"   Video date: {video_date}")
        print(f"   Frames extracted: {stats['total_frames_extracted']:,}")
        print(f"   Frames with captions: {stats['frames_with_captions']:,}")
        print(f"   Captions before dedupe: {stats.get('captions_before_dedupe', stats['frames_with_captions']):,}")
        print(f"   Unique embeddings: {stats.get('embeddings_generated', 0):,}")
        print(f"   ✅ Actually uploaded: {stats['embeddings_uploaded']:,}")
        print(f"   Processing time: {processing_time/60:.1f} minutes")
        print(f"\n   Frame reduction: {stats.get('frame_reduction_percent', 0):.1f}%")

        # Save video_name for next steps
        processed_video_name = video_name

    except Exception as e:
        print(f"\n❌ Error processing video: {e}")
        print("\nTroubleshooting tips:")
        print("- If GPU memory error: Restart runtime and try again")
        print("- If video format error: Convert video to MP4 format")
        import traceback
        traceback.print_exc()


Enter a name for this video (or press Enter for auto-name): testfr


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.



🎬 Processing video: testfr
📅 Video date: 2025-11-10
⏳ This will take a few minutes... Please wait.



preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

...

   ✓ Uploaded 2 vectors
   Sample caption: Backpack: An A man with a backpack walking down the street....

📁 Namespace: videos:2025-11-10:folder
   Uploading 1 vectors...


Uploading to Pinecone: 100%|██████████| 1/1 [00:00<00:00,  4.35it/s]


   ✓ Uploaded 1 vectors
   Sample caption: File: An a white refrigerator sitting on top of a wooden floor....

📁 Namespace: videos:2025-11-10:coat_jacket
   Uploading 1 vectors...


Uploading to Pinecone: 100%|██████████| 1/1 [00:00<00:00,  4.48it/s]


   ✓ Uploaded 1 vectors
   Sample caption: Coat Jacket: An A man in a red jacket is walking down the street....

✅ UPLOAD COMPLETE: 6 vectors uploaded for 'testfr'
   Sample vector IDs: e6d072b6ba7b_obj0_emb, 6d8f1a3acd55_obj1_emb, 6d8f1a3acd55_obj2_emb...



✅ VIDEO PROCESSING COMPLETE!

📊 Processing Statistics:
   Video name: testfr
   Video date: 2025-11-10
   Frames extracted: 6
   Frames with captions: 6
   Captions before dedupe: 6
   Unique embeddings: 6
   ✅ Actually uploaded: 6
   Processing time: 1.5 minutes

   Frame reduction: 0.0%


## Step 9: Search Your Video!

Now you can search for content using natural language queries.

**Example queries:**
- "person walking"
- "black bag"
- "someone talking on phone"
- "car driving"
- "red shirt"

The system will return timestamps where that content appears!

In [23]:
# Single search query with optional namespace filtering
query = input("🔍 Enter your search query: ")

# Optional: Search in specific object namespace
if use_object_detection:
    print("\n📁 Available namespaces:")
    print("   - backpack, bag, duffel_bag")
    print("   - laptop, tablet")
    print("   - helmet, bottle, folder, umbrella")
    print("   - coat_jacket, suitcase_luggage")
    print("   - (leave empty to search all)")

    namespace_filter = input("\nFilter by namespace (optional): ").strip().lower()
else:
    namespace_filter = ""

print(f"\nSearching for: '{query}'...")
if namespace_filter:
    print(f"Namespace filter: {namespace_filter}")
print("=" * 60)

# Perform search with namespace if specified
if namespace_filter and use_object_detection:
    # Search in specific namespace using Pinecone directly
    query_embedding = engine.embedding_generator.encode_query(query)
    search_results = engine.pinecone_manager.query(
        query_vector=query_embedding,
        top_k=5,
        namespace=namespace_filter,
        include_metadata=True
    )

    # Format results
    results = []
    for result in search_results:
        formatted_result = {
            "timestamp": result.timestamp,
            "caption": result.caption,
            "similarity_score": result.score,
            "frame_id": result.frame_id,
            "video_name": result.video_name,
            "time_formatted": engine._format_timestamp(result.timestamp)
        }
        results.append(formatted_result)
else:
    # Standard search across all namespaces
    results = engine.search(
        query=query,
        top_k=5,
        similarity_threshold=0.5
    )

if results:
    print(f"\n✅ Found {len(results)} results:\n")

    for i, result in enumerate(results, 1):
        print(f"{i}. ⏱️ Timestamp: {result['time_formatted']}")
        print(f"   📝 Caption: {result['caption']}")
        print(f"   📊 Confidence: {result['similarity_score']:.1%}")
        print(f"   🎥 Video: {result['video_name']}")
        print()
else:
    print("\n❌ No results found. Try:")
    print("   - Different search terms")
    print("   - More general queries")
    print("   - Lowering the similarity threshold")
    if namespace_filter:
        print(f"   - Searching without namespace filter (currently: {namespace_filter})")

🔍 Enter your search query: bag

📁 Available namespaces:
   - backpack, bag, duffel_bag
   - laptop, tablet
   - helmet, bottle, folder, umbrella
   - coat_jacket, suitcase_luggage
   - (leave empty to search all)

Filter by namespace (optional):  backpack

Searching for: 'bag'...
Namespace filter: backpack

✅ Found 2 results:

1. ⏱️ Timestamp: 00:03.80
   📝 Caption: Backpack: An A person with a backpack on their back.
   📊 Confidence: 81.1%
   🎥 Video: test

2. ⏱️ Timestamp: 00:00.03
   📝 Caption: Backpack: A The back of a backpack on a green field.
   📊 Confidence: 80.0%
   🎥 Video: test



## Step 10: View Namespace Statistics (Object Detection Mode)

If you used object detection mode, check how vectors are distributed across namespaces.

In [22]:
# View namespace distribution
if use_object_detection:
    stats = engine.get_index_stats()

    print("📊 NAMESPACE DISTRIBUTION")
    print("=" * 60)

    if 'namespaces' in stats and stats['namespaces']:
        print(f"\nTotal vectors: {stats['total_vectors']:,}")
        print(f"\nVectors by object category:")

        # Sort namespaces by vector count
        namespace_items = []
        for namespace, ns_info in stats['namespaces'].items():
            vector_count = ns_info.vector_count if hasattr(ns_info, 'vector_count') else ns_info
            namespace_items.append((namespace, vector_count))

        namespace_items.sort(key=lambda x: x[1], reverse=True)

        for namespace, count in namespace_items:
            # Format namespace name for display
            display_name = namespace.replace('_', ' ').title()
            percentage = (count / stats['total_vectors'] * 100) if stats['total_vectors'] > 0 else 0
            bar_length = int(percentage / 2)  # Scale to 50 chars max
            bar = "█" * bar_length

            print(f"   {display_name:20s} │ {bar:50s} {count:4d} ({percentage:5.1f}%)")
    else:
        print("No namespace data available yet. Process a video first.")

    print("=" * 60)
else:
    print("⚠️ Namespace statistics only available in object detection mode")
    print("   Re-run with use_object_detection = True to see namespace breakdown")

📊 NAMESPACE DISTRIBUTION

Total vectors: 256

Vectors by object category:
                        │ █████████████████████████████████████████████       235 ( 91.8%)
   Coat Jacket          │                                                       3 (  1.2%)
   Backpack             │                                                       2 (  0.8%)
   Videos:2025-11-10:Backpack │                                                       2 (  0.8%)
   Duffel Bag           │                                                       2 (  0.8%)
   Folder               │                                                       2 (  0.8%)
   Videos:2025-11-10:Duffel Bag │                                                       2 (  0.8%)
   Videos:2025-11-06:Duffel Bag │                                                       2 (  0.8%)
   Videos:2025-11-06:Backpack │                                                       2 (  0.8%)
   Videos:2025-11-10:Coat Jacket │                                             

## Step 11: Date-Based Search

Search videos from specific dates or date ranges for faster, more targeted results.

In [24]:
# Date-based search
print("🔍 DATE-BASED VIDEO SEARCH")
print("=" * 60)

# Show available dates
available_dates = engine.get_available_dates()

if available_dates:
    print(f"\n📅 Videos available for {len(available_dates)} dates:")
    for d in available_dates[:10]:  # Show first 10
        print(f"   - {d}")
    if len(available_dates) > 10:
        print(f"   ... and {len(available_dates) - 10} more dates")
else:
    print("\n⚠️ No videos with dates found in index")
    print("   Make sure you processed videos with date information")

# Get search query
print("\n" + "=" * 60)
query = input("Enter your search query: ")

# Date filtering options
print("\n📆 Date Filter Options:")
print("1. Search specific date (YYYY-MM-DD)")
print("2. Search date range (YYYY-MM-DD to YYYY-MM-DD)")
print("3. Search all dates (no filter)")

filter_option = input("\nSelect option (1/2/3, default=3): ").strip() or "3"

date_filter = None
start_date = None
end_date = None

if filter_option == "1":
    # Single date
    date_input = input("Enter date (YYYY-MM-DD): ").strip()
    if date_input in available_dates:
        date_filter = date_input
        print(f"✅ Searching only videos from {date_filter}")
    else:
        print(f"⚠️ Date {date_input} not found. Searching all dates.")

elif filter_option == "2":
    # Date range
    start_date = input("Start date (YYYY-MM-DD): ").strip()
    end_date = input("End date (YYYY-MM-DD): ").strip()
    print(f"✅ Searching videos from {start_date} to {end_date}")

# Optional category filter
if use_object_detection:
    print("\n📁 Category Filter (optional):")
    print("   backpack, bag, duffel_bag, laptop, tablet,")
    print("   helmet, bottle, folder, umbrella, coat_jacket, suitcase_luggage")
    namespace_filter = input("Filter by category (or press Enter to skip): ").strip().lower() or None
else:
    namespace_filter = None

print("\n" + "=" * 60)
print(f"Searching for: '{query}'")
if date_filter:
    print(f"Date: {date_filter}")
elif start_date and end_date:
    print(f"Date range: {start_date} to {end_date}")
if namespace_filter:
    print(f"Category: {namespace_filter}")
print("=" * 60)

# Perform search
try:
    if start_date and end_date:
        # Date range search
        results = engine.search_by_date_range(
            query=query,
            start_date=start_date,
            end_date=end_date,
            top_k=10,
            namespace_filter=namespace_filter
        )
    else:
        # Single date or no date filter
        results = engine.search(
            query=query,
            top_k=10,
            date_filter=date_filter,
            namespace_filter=namespace_filter
        )

    if results:
        print(f"\n✅ Found {len(results)} results:\n")

        for i, result in enumerate(results, 1):
            video_date_display = result.get('video_date', 'unknown')
            print(f"{i}. 📅 {video_date_display} | ⏱️ {result['time_formatted']}")
            print(f"   📝 {result['caption']}")
            print(f"   📊 Confidence: {result['similarity_score']:.1%}")
            print(f"   🎥 Video: {result['video_name']}")
            print()
    else:
        print("\n❌ No results found.")
        print("\nTry:")
        print("   - Different search terms")
        print("   - Broader date range")
        print("   - Removing filters")

except Exception as e:
    print(f"\n❌ Search error: {e}")
    import traceback
    traceback.print_exc()


🔍 DATE-BASED VIDEO SEARCH

📅 Videos available for 2 dates:
   - 2025-11-06
   - 2025-11-10



KeyboardInterrupt: Interrupted by user

In [ ]:
# Initialize Enhanced Search with Query Understanding
print("🚀 INITIALIZING ENHANCED SEARCH ENGINE")
print("=" * 80)

# Import the new modules
from enhanced_search import EnhancedSearch
from query_understanding import QueryUnderstanding

# Create enhanced search instance
enhanced = EnhancedSearch(engine)

print("✅ Enhanced search engine initialized!")
print("\n📚 New capabilities:")
print("   • Query understanding (entity/action/attribute extraction)")
print("   • Synonym-based query expansion")
print("   • Intent-aware reranking")
print("   • Hybrid semantic + keyword search")
print("   • Temporal query parsing")
print("   • Negation handling")

enhanced search

In [ ]:
# Test enhanced search with query understanding
print("🔍 ENHANCED SEARCH TEST")
print("=" * 80)

# Get search query
query = input("\n🔍 Enter your search query: ")

print(f"\nSearching with enhanced understanding...")
print("=" * 80)

# Perform enhanced search with explanation
results = enhanced.search_with_understanding(
    query=query,
    top_k=5,
    use_query_expansion=True,
    use_reranking=True,
    explain=True
)

# Display explanation
if 'explanation' in results:
    print(f"\n💡 QUERY ANALYSIS:")
    print(f"   {results['explanation']}")

# Display query expansions
if 'query_expansions' in results and len(results['query_expansions']) > 1:
    print(f"\n🔄 QUERY VARIATIONS ({len(results['query_expansions'])}):")
    for i, exp_q in enumerate(results['query_expansions'][:3], 1):
        print(f"   {i}. {exp_q}")

print(f"\n🎯 SEARCH STRATEGY:")
print(f"   Intent: {results['intent']}")
print(f"   Searched namespaces: {', '.join(results['searched_namespaces'])}")
print(f"   Total results found: {results['total_results']}")

# Display results
if results['results']:
    print("\n" + "=" * 80)
    print(f"✅ TOP {len(results['results'])} RESULTS:")
    print("=" * 80)
    
    for i, result in enumerate(results['results'], 1):
        print(f"\n{i}. ⏱️  Timestamp: {result['time_formatted']}")
        print(f"   📝 Caption: {result['caption']}")
        print(f"   📊 Score: {result['similarity_score']:.1%}")
        
        # Show additional scoring info if available
        if 'original_score' in result:
            boost = ((result['similarity_score'] - result['original_score']) / result['original_score'] * 100)
            if boost > 1:
                print(f"      ├─ Original: {result['original_score']:.1%}")
                print(f"      └─ Boosted: {result['similarity_score']:.1%} (+{boost:.0f}%)")
        
        if 'num_query_hits' in result:
            print(f"   🎯 Matched {result['num_query_hits']} query variations")
        
        print(f"   🎥 Video: {result['video_name']}")
else:
    print("\n❌ No results found")
    print("\n💡 Try:")
    print("   • More general terms")
    print("   • Different synonyms")

print("\n" + "=" * 80)

original vs enhanced

In [ ]:
# Compare original search vs enhanced search
print("⚖️  COMPARISON: ORIGINAL vs ENHANCED SEARCH")
print("=" * 80)

# Get search query
query = input("\n🔍 Enter a search query to compare: ")

print(f"\nRunning both search methods for: '{query}'")
print("=" * 80)

# 1. Original search
print("\n📌 ORIGINAL SEARCH (baseline):")
print("-" * 60)

original_results = engine.search(
    query=query,
    top_k=5,
    similarity_threshold=0.5
)

print(f"Results: {len(original_results)}")
if original_results:
    for i, r in enumerate(original_results[:3], 1):
        print(f"{i}. {r['time_formatted']} - {r['caption'][:60]}... ({r['similarity_score']:.1%})")

# 2. Enhanced search
print("\n🚀 ENHANCED SEARCH (with query understanding):")
print("-" * 60)

enhanced_results = enhanced.search_with_understanding(
    query=query,
    top_k=5,
    use_query_expansion=True,
    use_reranking=True,
    explain=False
)

print(f"Results: {len(enhanced_results['results'])}")
print(f"Intent: {enhanced_results['intent']}")
print(f"Searched: {', '.join(enhanced_results['searched_namespaces'])}")

if enhanced_results['results']:
    for i, r in enumerate(enhanced_results['results'][:3], 1):
        boost_info = ""
        if 'boosted_score' in r and 'original_score' in r:
            boost_pct = ((r['boosted_score'] - r['original_score']) / r['original_score'] * 100)
            if boost_pct > 1:
                boost_info = f" [+{boost_pct:.0f}%]"
        
        print(f"{i}. {r['time_formatted']} - {r['caption'][:60]}... ({r['similarity_score']:.1%}{boost_info})")

# Summary
print("\n" + "=" * 80)
print("📊 COMPARISON SUMMARY:")
print(f"   Original results: {len(original_results)}")
print(f"   Enhanced results: {len(enhanced_results['results'])}")

if len(enhanced_results['results']) > len(original_results):
    print(f"   ✅ Enhanced found {len(enhanced_results['results']) - len(original_results)} more results")
elif len(enhanced_results['results']) < len(original_results):
    print(f"   ℹ️  Enhanced is more selective (higher quality)")
else:
    print(f"   ℹ️  Same count, but potentially better ranked")

print("=" * 80)

hybrid search

In [ ]:
# Test hybrid search (semantic + keyword matching)
print("🔬 HYBRID SEARCH TEST")
print("=" * 80)
print("\nHybrid search combines:")
print("  • Semantic understanding (meaning-based)")
print("  • Keyword matching (exact word overlap)")

# Get search query
query = input("\n🔍 Enter your search query: ")

# Allow user to adjust weights
print("\n⚖️  Adjust search weights (or press Enter for defaults):")
sem_weight = input("   Semantic weight (default 0.7): ").strip()
key_weight = input("   Keyword weight (default 0.3): ").strip()

semantic_weight = float(sem_weight) if sem_weight else 0.7
keyword_weight = float(key_weight) if key_weight else 0.3

print(f"\nSearching with {semantic_weight:.0%} semantic + {keyword_weight:.0%} keyword...")
print("=" * 80)

# Perform hybrid search
results = enhanced.hybrid_search(
    query=query,
    top_k=5,
    semantic_weight=semantic_weight,
    keyword_weight=keyword_weight
)

# Display results
if results['results']:
    print(f"\n✅ Found {len(results['results'])} results:\n")
    
    for i, result in enumerate(results['results'], 1):
        print(f"{i}. ⏱️  Timestamp: {result['time_formatted']}")
        print(f"   📝 Caption: {result['caption']}")
        
        # Show score breakdown
        if 'semantic_score' in result and 'keyword_score' in result:
            print(f"   📊 Scores:")
            print(f"      ├─ Semantic: {result['semantic_score']:.1%}")
            print(f"      ├─ Keyword:  {result['keyword_score']:.1%}")
            print(f"      └─ Hybrid:   {result['hybrid_score']:.1%}")
        else:
            print(f"   📊 Score: {result['similarity_score']:.1%}")
        
        print(f"   🎥 Video: {result['video_name']}")
        print()
else:
    print("\n❌ No results found")

print("=" * 80)

temporal query search

In [ ]:
# Test temporal query understanding
print("⏰ TEMPORAL QUERY TEST")
print("=" * 80)
print("\nQueries with time references are automatically parsed:\n")

temporal_queries = [
    "backpack from yesterday",
    "bag from today",
    "recent duffel bag",
    "bag from 2025-11-10"
]

print("📝 Example temporal queries:")
for i, tq in enumerate(temporal_queries, 1):
    print(f"   {i}. {tq}")

print("\n" + "-" * 60)
choice = input("\nEnter a number (1-4) or type your own query: ").strip()

# Get the query
if choice.isdigit() and 1 <= int(choice) <= len(temporal_queries):
    query = temporal_queries[int(choice) - 1]
else:
    query = choice

print(f"\n⏳ Analyzing temporal context...")
print("=" * 80)

# Analyze query
qu = QueryUnderstanding()
expanded = qu.understand_query(query)

# Show temporal understanding
if expanded.structured_intent.temporal_context:
    print("\n📅 TEMPORAL CONTEXT DETECTED:")
    temporal = expanded.structured_intent.temporal_context
    
    if 'keyword' in temporal:
        print(f"   Keyword: {temporal['keyword']}")
    
    if 'specific_date' in temporal:
        print(f"   Specific date: {temporal['specific_date']}")
    
    if 'relative_time' in temporal and temporal['relative_time']:
        rel_time = temporal['relative_time']
        print(f"   Date range: {rel_time['start']} to {rel_time['end']}")
else:
    print("\n⚠️  No temporal context detected in query")

# Perform search
print("\n🔍 Searching...")
results = enhanced.search_with_understanding(
    query=query,
    top_k=5,
    use_query_expansion=True,
    use_reranking=True
)

# Display results
if results['results']:
    print(f"\n✅ Found {len(results['results'])} results:\n")
    
    for i, result in enumerate(results['results'], 1):
        print(f"{i}. ⏱️  Timestamp: {result['time_formatted']}")
        print(f"   📝 Caption: {result['caption']}")
        print(f"   📊 Score: {result['similarity_score']:.1%}")
        print(f"   🎥 Video: {result['video_name']}")
        print()
else:
    print("\n❌ No results found for this date/time")

print("=" * 80)

## Step 12: Batch Search (Multiple Queries)

Search for multiple things at once!

In [ ]:
# Define multiple queries
queries = [
    "person walking",
    "someone sitting",
    "black bag",
    "outdoor scene",
    "person talking"
]

print("🔍 Running batch search...\n")
print("=" * 60)

batch_results = engine.batch_search(queries, top_k=3)

for query, results in batch_results.items():
    print(f"\n📌 Query: '{query}'")
    print(f"   Found {len(results)} results")

    if results:
        for result in results[:2]:  # Show top 2
            print(f"   └─ {result['time_formatted']} - {result['caption'][:50]}... ({result['similarity_score']:.0%})")
    else:
        print("   └─ No results")

print("\n" + "=" * 60)

## Step 13: Advanced Search with Filters

Search with additional filters:
- Filter by specific video
- Search within time range
- Adjust confidence threshold

In [ ]:
# Advanced search example
query = input("Enter search query: ")

# Optional: Filter by time window (in seconds)
use_time_filter = input("Filter by time range? (y/n): ").lower() == 'y'

time_window = None
if use_time_filter:
    start_time = float(input("Start time (seconds): "))
    end_time = float(input("End time (seconds): "))
    time_window = (start_time, end_time)

# Optional: Filter by video name
video_filter = None
if 'processed_video_name' in locals():
    filter_video = input(f"Search only in '{processed_video_name}'? (y/n): ").lower() == 'y'
    if filter_video:
        video_filter = processed_video_name

# Perform search
print(f"\n🔍 Searching with filters...")
results = engine.search(
    query=query,
    top_k=10,
    similarity_threshold=0.4,  # Lower threshold for more results
    video_filter=video_filter,
    time_window=time_window
)

print(f"\n✅ Found {len(results)} results:\n")
for i, result in enumerate(results, 1):
    print(f"{i}. {result['time_formatted']} - {result['caption'][:60]}... ({result['similarity_score']:.1%})")

## Step 14: Interactive Search Interface

Interactive search with continuous queries.

In [ ]:
print("🎯 INTERACTIVE VIDEO SEARCH")
print("=" * 60)
print("Enter your search queries (type 'quit' to exit)\n")

while True:
    query = input("\n🔍 Search: ").strip()

    if query.lower() in ['quit', 'exit', 'q']:
        print("\n👋 Goodbye!")
        break

    if not query:
        continue

    results = engine.search(query, top_k=5)

    if results:
        print(f"\n✅ Found {len(results)} results:")
        for i, result in enumerate(results, 1):
            score_emoji = "🟢" if result['similarity_score'] > 0.7 else "🟡" if result['similarity_score'] > 0.5 else "🟠"
            print(f"\n{i}. {score_emoji} {result['time_formatted']} ({result['similarity_score']:.0%})")
            print(f"   {result['caption']}")
    else:
        print("\n❌ No results found. Try a different query.")

## Step 15: Troubleshooting - Verify Pinecone Upload

If vectors aren't showing up in Pinecone, use this diagnostic cell.

In [ ]:
# Diagnostic script to verify Pinecone upload
print("🔍 PINECONE UPLOAD DIAGNOSTICS")
print("=" * 80)

# 1. Check Pinecone connection
try:
    stats = engine.get_index_stats()
    print(f"\n✅ Connected to Pinecone index: {stats['index_name']}")
    print(f"   Total vectors: {stats['total_vectors']:,}")
    print(f"   Dimension: {stats['dimension']}")
    print(f"   Metric: {stats['metric']}")

    # 2. Check namespaces
    if 'namespaces' in stats and stats['namespaces']:
        print(f"\n📁 Namespaces found: {len(stats['namespaces'])}")
        for ns, info in stats['namespaces'].items():
            count = info.vector_count if hasattr(info, 'vector_count') else info
            print(f"   - {ns}: {count} vectors")
    else:
        print("\n⚠️  No namespaces found in index")
        print("   This means no vectors have been uploaded yet, OR")
        print("   vectors were uploaded to default namespace")

    # 3. Try querying default namespace
    print(f"\n🔎 Testing query on default namespace...")
    test_query = "backpack"
    query_emb = engine.embedding_generator.encode_query(test_query)
    results_default = engine.pinecone_manager.query(
        query_vector=query_emb,
        top_k=5,
        namespace=""  # Default namespace
    )
    print(f"   Default namespace results: {len(results_default)}")

    # 4. Try querying specific namespace (if using object detection)
    if use_object_detection:
        print(f"\n🔎 Testing query on 'backpack' namespace...")
        results_backpack = engine.pinecone_manager.query(
            query_vector=query_emb,
            top_k=5,
            namespace="backpack"
        )
        print(f"   Backpack namespace results: {len(results_backpack)}")

    # 5. Check if vectors were actually created
    if hasattr(engine, 'processed_frames') and engine.processed_frames:
        print(f"\n📊 Last processing session:")
        print(f"   Embedded frames created: {len(engine.processed_frames)}")
        if engine.processed_frames:
            sample = engine.processed_frames[0]
            print(f"   Sample embedding ID: {sample.embedding_id}")
            print(f"   Sample caption: {sample.captioned_frame.caption[:70]}...")
            print(f"   Embedding dimension: {len(sample.embedding)}")
            print(f"   Expected dimension: {engine.embedding_generator.embedding_dim}")
    else:
        print(f"\n⚠️  No processed frames found in engine")

    # 6. Recommendations
    print(f"\n💡 RECOMMENDATIONS:")
    if stats['total_vectors'] == 0:
        print("   ❌ No vectors in Pinecone index")
        print("   → Check if upload actually succeeded (look for error messages above)")
        print("   → Verify Pinecone API key and host are correct")
        print("   → Try re-running the video processing step")
    else:
        print("   ✅ Vectors found in index!")
        print(f"   → You have {stats['total_vectors']} vectors")
        print("   → Try searching with the cells above")

except Exception as e:
    print(f"\n❌ Error during diagnostics: {e}")
    import traceback
    traceback.print_exc()

print("\n" + "=" * 80)